In [1]:
import gradio as gr
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai_tools import tool

#from docx import Document
from io import BytesIO
import base64

import pandas as pd
from IPython.display import Markdown, display

from dotenv import load_dotenv
#load_dotenv("./.env")
load_dotenv("/home/jovyan/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
os.environ["SLIP_USERNAME"] =os.environ.get("SLIP_USERNAME")
os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")
os.environ["SERPER_API_KEY"] = "25d43901c968b0f8830387b7ed72231673428b9c"

/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_openai import ChatOpenAI

In [3]:
# from llm_utils import get_llm
# llm = get_llm('gpt-4o-0513')

In [4]:
llm=ChatOpenAI(model='gpt-4o-0513')

In [5]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## DEFINE AGENTS

In [6]:
diagnostician = Agent(
    role="Medical Diagnostician",
    goal="Analyze patient symptoms and medical history to provide a preliminary diagnosis.",
    backstory="This agent specializes in diagnosing medical conditions based on patient-reported symptoms and medical history. It uses advanced algorithms and medical knowledge to identify potential health issues.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool],
    llm=llm
)

treatment_advisor = Agent(
    role="Treatment Advisor",
    goal="Recommend appropriate treatment plans based on the diagnosis provided by the Medical Diagnostician.",
    backstory="This agent specializes in creating treatment plans tailored to individual patient needs. It considers the diagnosis, patient history, and current best practices in medicine to recommend effective treatments.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool],
    llm=llm
)


## DEFINE TASKS 

In [7]:
def create_tasks(symptoms, medical_history):
    diagnose_task = Task(
        description=(
            "1. Analyze the patient's symptoms ({symptoms}) and medical history ({medical_history}).\n"
            "2. Provide a preliminary diagnosis with possible conditions based on the provided information.\n"
            "3. Limit the diagnosis to the most likely conditions."
        ),
        expected_output="A preliminary diagnosis with a list of possible conditions.",
        output_file="diagnosis.md",
        agent=diagnostician)

    treatment_task = Task(
        description=(
            "1. Based on the diagnosis, recommend appropriate treatment plans step by step.\n"
            "2. Consider the patient's medical history ({medical_history}) and current symptoms ({symptoms}).\n"
            "3. Provide detailed treatment recommendations, including medications, lifestyle changes, and follow-up care."
        ),
        expected_output="A comprehensive treatment plan tailored to the patient's needs.",
        output_file="treatment_plan.md",
        agent=treatment_advisor)
    
    return [diagnose_task, treatment_task]

In [8]:
# # Create Crew
# crew = Crew(
#     agents=[diagnostician, treatment_advisor],
#     tasks=[diagnose_task, treatment_task],
#     verbose=2
# )

In [9]:
# Function to run the crew and get results
def run_crew(symptoms, medical_history):
    tasks = create_tasks(symptoms, medical_history)
    crew = Crew(
        agents=[diagnostician, treatment_advisor],
        tasks=tasks,
        verbose=True
    )
    
    inputs = {"symptoms": symptoms, "medical_history": medical_history}
    result = crew.kickoff(inputs)
    
    # Read the output files
    with open("diagnosis.md", "r") as f:
        diagnosis = f.read()
    with open("treatment_plan.md", "r") as f:
        treatment_plan = f.read()
    
    return diagnosis, treatment_plan

In [10]:
# Gradio interface
def gradio_interface(symptoms, medical_history):
    diagnosis, treatment_plan = run_crew(symptoms, medical_history)
    return diagnosis, treatment_plan

agent = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Symptoms"),
        gr.Textbox(label="Medical History")
    ],
    outputs=[
        gr.Markdown(label="Diagnosis"),
        gr.Markdown(label="Treatment Plan")
    ],
    title="Medical Diagnosis and Treatment Recommendation",
    description="Enter patient symptoms and medical history to get a diagnosis and treatment plan."
)


In [11]:
agent.launch(share=True)

which: no node in (/home/jovyan/envs/llm_agents/bin:/miniconda3/condabin:/opt/mssql-tools/bin:/usr/local/apache-jmeter-5.4.3/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/miniconda3/bin:/usr/lib/spark:/usr/lib/spark/bin:/usr/lib/jvm/java-1.8.0/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://34165ccabab6ea1bf1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
# inputs={"symptoms":"cold & fever",
#        "medical_history": "jaundice"}

In [13]:
# result = crew.kickoff(inputs)

In [14]:
#display(Markdown("./treatment_plan.md"))

 [2024-12-13 09:15:31][DEBUG]: == Working Agent: Medical Diagnostician
 [2024-12-13 09:15:31][INFO]: == Starting Task: 1. Analyze the patient's symptoms (fever) and medical history (cold).
2. Provide a preliminary diagnosis with possible conditions based on the provided information.
3. Limit the diagnosis to the most likely conditions.


> Entering new CrewAgentExecutor chain...
Given the patient's symptoms (fever) and medical history (cold), I need to analyze these details to provide a preliminary diagnosis. I will start by researching common conditions that can cause fever and are associated with a history of cold.

Action: Search the internet
Action Input: {"search_query": "common conditions causing fever with a history of cold"}
 

I encountered an error while trying to use the tool. This was the error: StructuredTool._run() missing 1 required keyword-only argument: 'config'.
 Tool Search the internet accepts these inputs: Search the internet(search_query: 'string') - A tool that c